# Control variates, antithetic variates, QMC


In [1]:
import numpy as np

In [10]:
def function(c, d, N):
    prop=0
    for i in range(N):
        X = np.random.normal(0, 1, d)
        if abs(np.prod(X))<=c:
            prop+=1
    return prop/N

In [46]:
function(.5, 3, 100)

0.74

In [5]:
np.prod(x)

0.005699945829662618